# Part 1: PageRank

## A Twitter-Mentioning Graph

* The edges are binary and directed. If Bob mentions Alice once, in 10 tweets, or 10 times in one tweet, there is an edge from Bob to Alice, but there is not an edge from Alice to Bob.
* If a user mentions herself, ignore it.
* Correctly parsing @mentions in a tweet is error-prone. Use the entities field.
* Later you will need to implement the PageRank algorithm on the graph you build here.


In [3]:
import json
import numpy as np
# Here define your function for building the graph by parsing the input file of tweets
# Insert as many cells as you want

def FileParser(filename):
    tweet_graph ={}            #Each (key,value) pair represents an edge from 'key' to 'pair'
    
    m=0 
    with open(filename) as myfname:
        m+=1 
        for line in myfname:
            temp_data= json.loads(line)
            id_current_user = temp_data["user"]["id"]
            if id_current_user not in tweet_graph:
                tweet_graph[id_current_user] = set()

            
            for index in range(len(temp_data["entities"]["user_mentions"])):
                id_ref_user = temp_data["entities"]["user_mentions"][index]["id"]
                if id_current_user != id_ref_user:               #to avoid self- loops (self- referencing)
                    tweet_graph[id_current_user].add(id_ref_user)
            
            #delete the "user" node from graph if it has no out-edges (i.e., it is not referring to anyone)
            if len(tweet_graph[id_current_user]) is 0:
                del tweet_graph[id_current_user]
            

    
    ##--------------------------------------------------------------------------------------
    #step-2 : id_lookup_dict creation: this will be useful for "pagerank" calculation later
    #id_lookup_dict - contains a simple integer 'value' starting from 0 for each 'userID' as 'key'
    # len(id_lookup_dict) = total number of different "users"
    id_index = 0 
    id_lookup_dict={}
    for ele in tweet_graph:
        if ele not in id_lookup_dict:
            id_lookup_dict[ele] = index
            index+=1
        
        for neighbour in tweet_graph[ele]:
            if neighbour not in id_lookup_dict:
                id_lookup_dict[neighbour] = index
                index +=1

                
    ##-------------------------------------------------------------------------------------
    #step-3 : updating "tweet_graph" to include all the nodes 
    for ele in id_lookup_dict:
        if ele not in tweet_graph:
            tweet_graph[ele]= set()
    
    return (tweet_graph, id_lookup_dict)


    


In [4]:
# Call your function to print out the size of the graph, i.e., the number of nodes and edges
# How you maintain the graph is totaly up to you
# However, if you encounter any memory issues, we recommend you write the graph into a file, and load it later.
tweet_graph, id_lookup_dict = FileParser('pagerank.json')
tot_edges=0 

for node,edge in tweet_graph.iteritems():
    tot_edges+= len(edge )

print("No. of nodes = "+ str(len(tweet_graph)) + " and No. of edges = " +str(tot_edges))    
    
    


1
No. of nodes = 16430 and No. of edges = 24256


In [5]:
# Here add your code to implement a function called PageRanker
# Insert as many cells as you want


def PageRanker(tweet_graph, id_lookup_dict):  

    d = 0.9 
    N = len(id_lookup_dict)
    page_rank_vector = [(1-d)/float(N)] * N 
    last_page_rank_vector =None 
    diff_page_rank_value=  1
    
    while diff_page_rank_value >= 0.0001 :
        
        last_page_rank_vector, page_rank_vector = page_rank_vector, [(1-d)/float(N)]*N

        for ele in tweet_graph:
            ele_length = len(tweet_graph[ele])
            
            #nodes with no out-edges need not be considered in calculations
            if ele_length==0:
                continue
            ele_PR_value = last_page_rank_vector[id_lookup_dict[ele]]
            
            #'neighbor' has an in-link from 'ele'. so it will receive a fraction of 'ele's' PR score 
            for neighbor_id in tweet_graph[ele]: 
                neighbor_index= id_lookup_dict[neighbor_id]
                page_rank_vector[neighbor_index] += float(d*ele_PR_value)/float(ele_length)

        #Termination condition error calculation 
        diff_page_rank_value = 0 
        for i in range(len(page_rank_vector)):
            diff_page_rank_value += abs(page_rank_vector[i] - last_page_rank_vector[i])
        sorted_page_rank_vec_index = sorted(range(len(page_rank_vector)), key=lambda k: page_rank_vector[k], reverse=True)
        sorted_last_page_rank_vec_index= sorted(range(len(last_page_rank_vector)), key=lambda k: page_rank_vector[k], reverse=True)
        
    #normalizing PR score at the end     
    tot_score= 0 
    for score in page_rank_vector:
        tot_score += score
        
    for i in range(len(page_rank_vector)):
        page_rank_vector[i] /=tot_score
        
    return page_rank_vector
 

In [6]:
# Now let's call your function on the graph you've built. Output the results.

tweet_graph, id_lookup_dict = FileParser('pagerank.json')
page_rank_vector            = PageRanker(tweet_graph, id_lookup_dict)

#Sorting the page rank results and printing top 10 results as per the pagerank score calculated 
sort_page_rank_vec_index = sorted(range(len(page_rank_vector)), key=lambda k: page_rank_vector[k], reverse=True)
for i in sort_page_rank_vec_index[0:10]:
    for id,index in id_lookup_dict.iteritems():
        if index == i:
            print("user_id= " +str(id) + ", score= "+str(page_rank_vector[index]))
            break

user_id= 158314798, score= 0.00771549905257
user_id= 181561712, score= 0.00652134413971
user_id= 209708391, score= 0.00607439403128
user_id= 72064417, score= 0.00553688801417
user_id= 105119490, score= 0.00508362381201
user_id= 14268057, score= 0.0046790143692
user_id= 379961664, score= 0.00442468466829
user_id= 391037985, score= 0.00425692488481
user_id= 153074065, score= 0.00425357283333
user_id= 313525912, score= 0.00409497311133


NOTE: Done as a part of CS670 course assignment at Texas A&M University